# PCA
- Brigitte Hogan (bwh5v@virginia.edu) & Jason Tiezzi (jbt5am@virginia.edu)  
- DS 5001: Exploratory Text Analytics  
- April 2020  
---

<font color = gray>

## Overview

This notebook

1. Creates a reduced `TFIDF` table; that is, select only the top 5,000 most significant terms.

2. Performs PCA on the reduced `TFIDF` "by hand," i.e. create a covariance matrix of features, apply eigen-decomposition, select components, etc. In the process, generate `COMPS`, `LOADINGS`, and `DCM` tables from your results (as in the in-class example).

3. Using whatever visualization libraries you can*, inspect the first three components and answer the following questions:

    (1) What `LIB` feature (author or genre) does the first principal component (PC) separate?

    (2) Based on the first PC, what two novelists are most opposite to (distant from) each other?

    (3) Based on the second PC, what two novelists are most opposite to each other?

    (4) Based on the third PC, what two novelists are most opposite to each other?

    (5) Based on your knowledge of linguistic annotations, what implicit feature do you think accounts for the clear separation of novels in our data?

---
# Set Up

## Config

In [1]:
data_dir = '../../Notebooks/Homework/Homework07_bwh5v/HW_7_DATA/'

OHCO = ['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num'] # define OHCO
CHAPS = ['book', 'chapter'] # alternate OHCO

## Import

In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from scipy.linalg import norm
import plotly_express as px
import seaborn as sns
from scipy.linalg import eigh

In [3]:
sns.set(style='ticks')
%matplotlib inline

## Functions

### tfidf()

In [4]:
def tfidf (token, ohco, bag='CHAPS', count_method='n', item_type='term_id', tf_method='sum', idf_method='standard'):
    ## Arguments -----------------------------------------------------------------------------------------
    # token (pandas dataframe): must have term_str and term_id or stem_porter
    # bag (string) = OHCO_level, either - BOOKS, CHAPS, PARAS, SENTS
    # count_method (string): either 'n' (default) for n tokens/ regular or 'c' for distinct tokens/ binary
    # item_type (string): type of item to count, either 'term' for terms or 'stem' for stems
    # tf_method (string): tf method - sum (default), max, log, double_norm, raw, binary
    # idf_method (string): idf method - standard (default), max, or smooth
    
    ## Create OHCO Dictionary for Bag --------------------------------------------------------------------
    #OHCOdict = {
    #    "BOOKS": ['book_id'],
    #    "CHAPS": ['book_id', 'chap_num'],
    #    "PARAS": ['book_id', 'chap_num', 'para_num'],
    #    "SENTS": ['book_id', 'chap_num', 'para_num', 'sent_num']
    #    }
    OHCOdict = {
        "BOOKS": [ohco[0]],
        "CHAPS": [ohco[0], ohco[1]],
        "PARAS": [ohco[0], ohco[1], ohco[2]],
        "SENTS": [ohco[0], ohco[1], ohco[2], ohco[3]]
        }
    theBag = OHCOdict[bag]
    
    ## Create Bag-of-Words/Stems -------------------------------------------------------------------------
    BOW = token.groupby(theBag + [item_type])[item_type].count().to_frame().rename(columns={item_type:'n'})
    
    ## Add Binary Count Column ---------------------------------------------------------------------------
    BOW['c'] = BOW.n.astype('bool').astype('int')
    
    ## Create Document Term Frequency Matrix -------------------------------------------------------------
    #DTCM = BOW[count_method].unstack().fillna(0)
    DTCM = BOW[count_method].unstack(fill_value=0) # Raf's
    
    ## Compute TF ----------------------------------------------------------------------------------------
    if tf_method == 'sum':
        TF = DTCM.T / DTCM.T.sum()
    elif tf_method == 'max':
        TF = DTCM.T / DTCM.T.max()
    elif tf_method == 'log':
        TF = np.log10(1 + DTCM.T)
    elif tf_method == 'raw':
        TF = DTCM.T
    elif tf_method == 'double_norm':
        tf_norm_k = .5
        TF = DTCM.T / DTCM.T.max()
        TF = tf_norm_k + (1 - tf_norm_k) * TF[TF > 0]
    elif tf_method == 'binary':
        TF = DTCM.T.astype('bool').astype('int')
    
    ## Compute IDF ---------------------------------------------------------------------------------------
    N = DTCM.shape[0]
    DF = DTCM[DTCM > 0].count()   
    
    if idf_method == 'standard':
        IDF = np.log10(N / DF)
    elif idf_method == 'max':
        IDF = np.log10(DF.max() / DF) 
    elif idf_method == 'smooth':
        IDF = np.log10((1 + N) / (1 + DF)) + 1 

    ## Compute TFIDF -------------------------------------------------------------------------------------
    TFIDF = TF.T * IDF
    
    return TFIDF
    

### get_tfidf()

In [5]:
def get_tfidf(TOKEN, bag=CHAPS, count_method='n', tf_method='sum', idf_method='standard', item_type='term_id'):
    
    # Create bag of items (terms or stems)
    BOW = TOKEN.groupby(bag + [item_type])[item_type].count().to_frame().rename(columns={item_type:'n'})

    # Add binary count column
    BOW['c'] = BOW.n.astype('bool').astype('int')
    
    # Create document-term matrix
    DTCM = BOW[count_method].unstack(fill_value=0)#.astype('int')
    
    # Compute TF
    if tf_method == 'sum':
        TF = DTCM.T / DTCM.T.sum()
    elif tf_method == 'max':
        TF = DTCM.T / DTCM.T.max()
    elif tf_method == 'log':
        TF = np.log10(1 + DTCM.T)
    elif tf_method == 'raw':
        TF = DTCM.T
    elif tf_method == 'double_norm':
        TF = DTCM.T / DTCM.T.max()
        TF = tf_norm_k + (1 - tf_norm_k) * TF[TF > 0] 
    elif tf_method == 'binary':
        TF = DTCM.T.astype('bool').astype('int')  
    
    # Compute IDF
    N = DTCM.shape[0]
    DF = DTCM[DTCM > 0].count()
    if idf_method == 'standard':
        IDF = np.log10(N / DF)
    elif idf_method == 'max':
        IDF = np.log10(DF.max() / DF) 
    elif idf_method == 'smooth':
        IDF = np.log10((1 + N) / (1 + DF)) + 1
    
    # Compute TF-IDF
    TFIDF = TF.T * IDF
    return TFIDF

## vis_pcs()

In [6]:
def vis_pcs(M, a, b, label='author', prefix='PC'):
    fig = px.scatter(M, prefix + str(a), prefix + str(b), 
                     color=label, 
                     hover_name='doc', 
                     marginal_x='box',
                     marginal_y='box',
                     width=1000, height = 600)
    fig.show()

---
# Prepare the Data

## Import Tables

In [7]:
LIB   = pd.read_csv(data_dir + 'novels-LIB.csv')                        # book, genre, author
VOCAB = pd.read_csv(data_dir + 'novels-VOCAB.csv').set_index('term_id') # term_id, term_str, tfidf_sum, n
TOKEN = pd.read_csv(data_dir + 'novels-TOKENS.csv')                     # OHCO, pos, term_str, term_id

## Format Tables

In [8]:
LIB = LIB.rename(columns={"book": "book_id"}).set_index('book_id')

In [9]:
LIB['genre_full'] = LIB.genre.replace({'d': 'detective', 'g': 'gothic fiction', 'nh': 'unknown'})

In [10]:
LIB.head()

,genre,author,genre_full
book_id,,,
secretadversary,d,christie,detective
styles,d,christie,detective
moonstone,d,collins,detective
adventures,d,doyle,detective
baskervilles,d,doyle,detective


In [11]:
TOKEN = TOKEN.rename(columns={"book": "book_id","chapter": "chap_num"}).set_index(OHCO)

# 1. Create reduced TFIDF Matrix

In [12]:
TFIDF1 = tfidf(TOKEN, ohco=OHCO, bag="CHAPS")
TFIDF2 = get_tfidf(TOKEN, bag=['book_id', 'chap_num'])
print(TFIDF1.equals(TFIDF2)) # True
print(VOCAB.tfidf_sum.round(12).equals(TFIDF1.sum().round(12)))
print(VOCAB.tfidf_sum.round(12).equals(TFIDF2.sum().round(12)))

True
True
True


In [13]:
TFIDF = TFIDF1

In [14]:
VOCAB.head()

,term_str,tfidf_sum,n
term_id,,,
0,a,0.000000,28533
1,aback,0.003732,9
2,abaft,0.000876,2
3,abandon,0.006993,44
4,abandoned,0.010044,68


### Select Top 5,000 Significant Words

#### Using tfidf_sum for significance measure

In [15]:
VOCAB = VOCAB.sort_values('tfidf_sum', ascending=False)[0:5000].sort_index()

In [16]:
TOKEN = TOKEN[TOKEN.term_id.isin(VOCAB.index)]

In [17]:
TFIDF = TFIDF.loc[:, TFIDF.columns.isin(VOCAB.index)]

In [18]:
TFIDF.shape  # reduced TFIDF, just significant vocab

(320, 5000)

# Pre-process TFIDF Matrices

In [ ]:
TFIDF.head()

## Normalize doc vector lengths

In [ ]:
TFIDF = TFIDF.apply(lambda x: x / norm(x, 2), 1) # L2 normalization

In [ ]:
TFIDF.head()

# Compute Covariance Matrix

In [ ]:
COV = TFIDF.cov()

In [ ]:
COV.iloc[:5,:10].style.background_gradient() # limit this so it doesn't crash your system

## Decompose the Matrix

In [ ]:
%time eig_vals, eig_vecs = eigh(COV)

## Convert eigen data to dataframes

In [ ]:
TERM_IDX = COV.index # for convenience

In [ ]:
EIG_VEC = pd.DataFrame(eig_vecs, index=TERM_IDX, columns=TERM_IDX)

In [ ]:
EIG_VAL = pd.DataFrame(eig_vals, index=TERM_IDX, columns=['eig_val'])
EIG_VAL.index.name = 'term_id'

In [ ]:
EIG_VEC.iloc[:5, :10].style.background_gradient()

In [ ]:
EIG_VAL.iloc[:5] # this is the ranking principal

# Select Principal Components

Associate each eigenvalue with its corresponding *column* in the eigenvalue matrix by transposing the  `EIG_VEC` dataframe.

## Combine eigenvalues and eignvectors

In [ ]:
EIG_PAIRS = EIG_VAL.join(EIG_VEC.T) # join into table

In [ ]:
EIG_PAIRS.head()                    # term_ids ~ components

## Compute and Show Explained Variance

We might have usd this value to sort our components.

In [ ]:
EIG_PAIRS['exp_var'] = np.round((EIG_PAIRS.eig_val / EIG_PAIRS.eig_val.sum()) * 100, 2)

In [ ]:
EIG_PAIRS.exp_var.sort_values(ascending=False).head().plot.bar(rot=45);

## Pick Top 3 Components

We pick these based on explained variance.

In [ ]:
COMPS = EIG_PAIRS.sort_values('exp_var', ascending=False).head(3).reset_index(drop=True)
COMPS.index.name = 'comp_id'
COMPS.index = ["PC{}".format(i) for i in COMPS.index.tolist()]

In [ ]:
COMPS # each term associated with component and weight

# Inspect terms associated with eigenvectors

In [ ]:
VOCAB.loc[[int(x) for x in EIG_PAIRS.sort_values('exp_var').head(10).index], 'term_str']

## Show Loadings

In [ ]:
LOADINGS = COMPS[TERM_IDX].T
LOADINGS.index.name = 'term_id'

In [ ]:
LOADINGS.head(20).style.background_gradient()

In [ ]:
LOADINGS['term_str'] = LOADINGS.apply(lambda x: VOCAB.loc[int(x.name)].term_str, 1)

In [ ]:
l0_pos = LOADINGS.sort_values('PC0', ascending=True).head(10).term_str.str.cat(sep=' ') # looking at max pos and neg for 1st three components
l0_neg = LOADINGS.sort_values('PC0', ascending=False).head(10).term_str.str.cat(sep=' ')
l1_pos = LOADINGS.sort_values('PC1', ascending=True).head(10).term_str.str.cat(sep=' ')
l1_neg = LOADINGS.sort_values('PC1', ascending=False).head(10).term_str.str.cat(sep=' ')
l2_pos = LOADINGS.sort_values('PC2', ascending=True).head(10).term_str.str.cat(sep=' ')
l2_neg = LOADINGS.sort_values('PC2', ascending=False).head(10).term_str.str.cat(sep=' ')

In [ ]:
print('Books PC0+', l0_pos)
print('Books PC0-', l0_neg)
print('Books PC1+', l1_pos)
print('Books PC1-', l1_neg)
print('Books PC2+', l2_pos)
print('Books PC2-', l2_neg)

# Project Docs onto New Subspace

Get Document-Component Matrix (DCM)

In [ ]:
DCM = TFIDF.dot(COMPS[TERM_IDX].T)

In [ ]:
DCM # each doc/chapter has distribution of components

#### Add Labels for Display

In [ ]:
LIB = LIB.reset_index()
LIB["title"] = LIB.book_id
LIB = LIB.set_index('book_id')
LIB.head()

In [ ]:
DCM = DCM.join(LIB[['author','genre_full','title']], on='book_id')

In [ ]:
DCM['doc'] = DCM.apply(lambda x: "{}-{}-{}".format(x.author, x.title, x.name[1]), 1)

In [ ]:
DCM.head()

In [ ]:
DCM.head(10).style.background_gradient() # Note: Components become features for VOCAB and DOC tables

# Visualize

## PC 0 and 1

In [ ]:
vis_pcs(DCM, 0, 1) # by author

vis_pcs(DCM, 0, 1, label='genre_full')

In [ ]:
#vis_pcs(DCM, 0, 1, label='title')

## PC 1 and 2

In [ ]:
vis_pcs(DCM, 1, 2) # by author

In [ ]:
vis_pcs(DCM, 1, 2, label='genre_full')

In [ ]:
#vis_pcs(DCM, 1, 2, label='title')

## PC 0 and 2

In [ ]:
vis_pcs(DCM, 0, 2) # author

In [ ]:
vis_pcs(DCM, 0, 2, label='genre_full')

In [ ]:
#vis_pcs(DCM, 0, 2, label='title')

---
## Results

**1. What `LIB` feature (author or genre) does the first principal component (PC) separate?**  
The first principal component (PC0) separates primarily on genre. The second principal component (PC1) does a better job of separating author. 

**2. Based on the first PC (PC0), what two novelists are most opposite to (distant from) each other?**  
Radcliffe & Christie

**3. Based on the second PC (PC1), what two novelists are most opposite to each other?**  
Austen & Christie

**4. Based on the third PC (PC2), what two novelists are most opposite to each other?**  
Collins & Austen

**5. Based on your knowledge of linguistic annotations, what implicit feature do you think accounts for the clear separation of novels in our data?**  
By looking at the loadings, it appears the novels are being separated by proper nouns, most of which are the names of the principal characters.